# Apprentissage profond, TD1: pratique basique de PyTorch

## 1. Apprentissage de fizz buzz

In [71]:
import numpy as np
import torch
from tqdm import tqdm

In [72]:
def fizz_buzz(i: int, prediction)-> list:
    return [ str (i) , "fizz" , "buzz" , " fizzbuzz "][ prediction ]

In [73]:
# creation verite terrain : [ number , "fizz" , "buzz" , "fizzbuzz"]
def fizz_buzz_encode (i) :
    if i % 3 ==  0 and i % 5 == 0:
        return 3
    if i % 3 == 0: # Multiples de 3 remplacés par "fizz"
        return 1
    if i % 5 == 0: # Multiples de 5 par buzz
        return 2
    
    return 0

In [74]:
fizz_buzz_encode(15)

3

### Modélisation des entrées et des données

In [75]:
NUM_DIGITS = 10
def binary_encode (i , num_digits ):
    return np.array ([ (i >> d) & 1 for d in range ( num_digits ) ]) # L'opérateur >> décale le nombre entier $i$ de $d$ positions vers la droite. L'opérateur & (AND binaire) compare les bits du résultat de la première opération avec 1

Création du corpus d'apprentissage:

In [76]:
# données d’entraînement (X) et labels (Y)
X_train = torch.FloatTensor ([ binary_encode(i , NUM_DIGITS ) for i in range ( 101 , 1024 )])
Y_train = torch.LongTensor ([ fizz_buzz_encode(i) for i in range (101 , 1024)]).squeeze()

# données de test
X_test = torch.FloatTensor ([ binary_encode(i , NUM_DIGITS ) for i in range (1 , 101)])

Création du model, de la loss et de l'optimizer

In [77]:
# Nombres de neurones dans la coucge cachée
HIDDEN_SIZE = 100

# Définition du MLP à une couche cachée (non linéaire: ReLu)
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS , HIDDEN_SIZE),
    torch.nn.ReLU(),
    torch.nn.Linear(HIDDEN_SIZE , 4)
)

On va utiliser la cross entropy comme loss en s'appuyant sur le principe du maximum de vraisemblance.

In [78]:
loss_fn = torch.nn.CrossEntropyLoss()

La cross entropy est implémenté dans pytorch aussi dans nnNLLLoss(), NLL est l'acronyme de negative log likelihood loss, elle est nommé ainsi car elle s'attends a recevoir des log-probabilités.

En pratique, l'implémentation de nn.CrossEntropyLoss() dans PyTorch est en réalité une combinaison de deux étapes :$$\text{nn.CrossEntropyLoss}(\text{Logits}) = \text{nn.NLLLoss}(\text{nn.LogSoftmax}(\text{Logits}))$$

**Avantage** : Elle est plus numériquement stable que de réaliser les deux étapes séparément, car les calculs de log-probabilités sont optimisés pour éviter les erreurs d'arrondi (comme les $\log(0)$ ou les exponentielles de très grands nombres).

On choisit ensuite de minimiser le coût par le gradient stochastique (SGD):

In [79]:
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

Le Learning Rate ($\text{lr}$ ou $\eta$) est un scalaire positif qui détermine la taille du pas effectué dans la direction opposée au gradient de la fonction de perte (loss function).Il agit comme un facteur de mise à l'échelle appliqué au gradient calculé sur le mini-batch actuel.

La règle de mise à jour pour un poids $W$ dans un $\text{SGD}$ est donnée par : $W_{\text{new}} = W_{\text{old}} - \eta \cdot \nabla J(W_{\text{old}})$

$W_{\text{new}}$ : Le nouveau poids après la mise à jour.$W_{\text{old}}$ : Le poids actuel.$\eta$ (ou $\text{lr}$) : Le Learning Rate.$\nabla J(W_{\text{old}})$ : Le gradient de la fonction de perte $J$ par rapport aux poids $W$ (indiquant la direction de la plus forte pente).

In [80]:
BATCH_SIZE = 128
raw_data_test = np.arange(1 , 101) # Valeurs de test
for epoch in tqdm(range(1000), desc="Training Epochs", unit="epoch"):
    for start in range(0 , len (X_train) , BATCH_SIZE ):
        end = start + BATCH_SIZE
        batch_X = X_train[start:end]
        batch_Y = Y_train[start:end]
        
        # Forward pass
        y_pred = model(batch_X)
        
        # Calcul de la loss
        loss = loss_fn(y_pred, batch_Y)
        
        # Optimisation
        optimizer.zero_grad()

        # Backward pass (retro-propagation)
        loss.backward()
        optimizer.step()
    
    # Calcul du coût (et affichage)
    loss = loss_fn(model(X_train), Y_train)
    if epoch % 100 == 0:
        tqdm.write(f"Epoch {epoch} - Loss: {loss.item()}")
    
    # Visualisation des résultats en cours d'apprentissage
    if epoch % 1000 == 0:
        Y_test_pred = model(X_test)
        val, idx = torch.max(Y_test_pred, 1)
        ii = idx.data.numpy()

        output = np.vectorize(fizz_buzz)(raw_data_test, ii)
        print(output)

Training Epochs:   3%|▎         | 28/1000 [00:00<00:07, 137.42epoch/s]

Epoch 0 - Loss: 1.196959376335144
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' 'fizz' '54' '55' '56' 'fizz' '58'
 '59' '60' 'fizz' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100']


Training Epochs:   7%|▋         | 69/1000 [00:00<00:06, 140.30epoch/s]


KeyboardInterrupt: 

In [ ]:
# Sortie finale après entraînement (calcul lisible)
Y_test_pred = model(X_test)
val, idx = torch.max(Y_test_pred, 1)
ii = idx.data.numpy()
output = np.vectorize(fizz_buzz)(raw_data_test, ii)
print("============= Final Output =============")
print(output)

# Sortie finiale (calcul plus compact des prédictions)
Y_test_pred = model(X_test)
predictions = zip(range(1, 101), list(Y_test_pred.max(1)[1].data.tolist()))
print("============= Final Output (compact) =============")
print([fizz_buzz(i, prediction) for i, prediction in predictions])

============= Final Output =============
['1' '2' 'fizz' '4' 'buzz' '6' '7' '8' 'buzz' '10' '11' 'fizz' '13' '14'
 '15' '16' '17' '18' '19' '20' 'fizz' '22' '23' '24' 'buzz' '26' '27' '28'
 '29' '30' '31' '32' '33' '34' 'buzz' '36' '37' '38' '39' 'buzz' '41'
 'fizz' '43' '44' ' fizzbuzz ' '46' '47' '48' '49' 'buzz' '51' '52' '53'
 '54' 'buzz' '56' '57' '58' '59' ' fizzbuzz ' '61' '62' 'fizz' '64' '65'
 'fizz' '67' '68' '69' '70' '71' 'fizz' '73' '74' 'fizz' '76' '77' 'fizz'
 '79' '80' '81' '82' '83' 'fizz' 'buzz' '86' 'fizz' '88' '89' ' fizzbuzz '
 '91' '92' 'fizz' '94' '95' 'fizz' '97' '98' '99' 'fizz']
============= Final Output (compact) =============
['1', '2', 'fizz', '4', 'buzz', '6', '7', '8', 'buzz', '10', '11', 'fizz', '13', '14', '15', '16', '17', '18', '19', '20', 'fizz', '22', '23', '24', 'buzz', '26', '27', '28', '29', '30', '31', '32', '33', '34', 'buzz', '36', '37', '38', '39', 'buzz', '41', 'fizz', '43', '44', ' fizzbuzz ', '46', '47', '48', '49', 'buzz', '51', '52', '5

Obtention du taux de classification sur les données de test

In [ ]:
gtY = np.array([fizz_buzz_encode(i) for i in range(1, 101)])
print("Accuracy:", np.mean(ii == gtY))

Accuracy: 0.75


Séparation des données d'entrainement pour monitorer l'apprentissage correctement:

In [ ]:
NUM_VAL = 100
p = np.random.permutation(range(len(X_train)))
X_train, Y_train = X_train[]